In [1]:
import tensorflow as tf
import keras
import random
from pathlib import Path 
from time import strftime 

In [2]:
# Load the dataset
(X_train_all, y_train_all), (X_test, y_test) = keras.datasets.cifar10.load_data()

X_valid, y_valid = X_train_all[-5000:], y_train_all[-5000:]
X_train, y_train = X_train_all[:-5000], y_train_all[:-5000]


In [ ]:
X_train.shape

In [3]:
print(X_train[0, 0:32, 0, 0]) # integers between zero and 255 

[ 59  16  25  33  50  71  97 115 137 154 154 145 142 158 145 148 149 147
 152 145 143 143 141 143 149 172 202 216 220 208 180 177]


In [4]:
X_train = (X_train / 255.)
X_valid = (X_valid / 255.) 
X_test = (X_test / 255.)

In [5]:
param_dict = {
    'learning_rate' : [0.0001, 0.0005, 0.001, 0.005, 0.01], 
    'batch_size' : [32, 64, 128], 
    'epochs' : [50, 100, 200], 
    'num_layers' : [1, 2, 3],
    'neurons' : [64, 128, 256]
}

In [7]:
## this is the function for the random model
def random_model(neurons=128, num_layers=2):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=[32, 32, 3]))
    model.add(tf.keras.layers.Flatten())
    num_dense = random.randint(1, 3)
    for _ in range(num_layers):
        model.add(tf.keras.layers.Dense(neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    return model

In [8]:
def get_hyperparams(param_dict):
    list = []
    for key, values in param_dict.items():
        dict = {key : random.choice(values) for key, values in param_dict.items()}
        return dict

In [9]:
def get_run_logdir(root_logdir="my_logs"):
    return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")

run_logdir = get_run_logdir()

In [12]:
results = []
num_trials = 1
for trial in range(num_trials):
    hps = get_hyperparams(param_dict)
    print(f"Hyperparams for trial {trial}: {hps}")
    
    model = random_model(neurons=hps['neurons'], num_layers=hps["num_layers"])
    optimizer = tf.keras.optimizers.Adam(learning_rate=hps["learning_rate"])
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    
    run_logdir = Path("my_logs") / f"run_{trial:03d}_{strftime('%Y%m%d_%H%M%S')}"
    tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
    
    history = model.fit(
        X_train, y_train, 
        epochs=hps['epochs'],
        validation_data=(X_valid, y_valid), 
        callbacks=[tensorboard_cb],
        batch_size=hps['batch_size'],
    )
    
    results.append({
        'hyperparams' : hps,
        'final_val_acc' : max(history.history['val_accuracy']),
        'final_train_acc' : max(history.history['accuracy']),
        'run_id' : trial
    })

Hyperparams for trial 0: {'learning_rate': 0.005, 'batch_size': 64, 'epochs': 200, 'num_layers': 1, 'neurons': 64}
Epoch 1/200
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.1026 - loss: 2.8194 - val_accuracy: 0.0976 - val_loss: 2.3029
Epoch 2/200
704/704 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0994 - loss: 2.3027 - val_accuracy: 0.0950 - val_loss: 2.3033
Epoch 3/200
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0968 - loss: 2.3032 - val_accuracy: 0.0958 - val_loss: 2.3028
Epoch 4/200
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0977 - loss: 2.3032 - val_accuracy: 0.0976 - val_loss: 2.3025
Epoch 5/200
704/704 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0982 - loss: 2.3030 - val_accuracy: 0.0958 - val_loss: 2.3031
Epoch 6/200
704/704 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0991 - loss: 2.3031 - val_accuracy: 0.1546 - val_loss: 2.2493
Epoch 7/200
704/704 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1552 - loss: 2.1911 - val_accuracy: 0.1758 -

In [13]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs

Reusing TensorBoard on port 6006 (pid 19981), started 1:15:07 ago. (Use '!kill 19981' to kill it.)